In [72]:
import ROOT as r
import coffea
import coffea.hist as hist
from coffea.nanoevents.methods import vector
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np

from pyjet import cluster
import awkward as ak
import mplhep

In [187]:
events_raw = uproot.open("~/eos/llp/HNL/HeavyNeutralLepton_Tree.root")['MuonSystem']
# events_raw = uproot.open("../ggH_HToSSTodddd_v2/HeavyNeutralLepton_Tree_45_7_1000.root")['MuonSystem']

df = coffea.processor.LazyDataFrame(events_raw,entrystop=10)
events = uproot.lazy(df._tree)

In [137]:
events = events[:2000]

In [204]:
events.jetMet_dPhi

<Array [-2.4, -999, -2.78, ... -999, -999] type='151269 * float32'>

In [188]:
llp_csc = ak.any(events.gLLP_csc,axis=1)
print("LLP decay inside CSC = ",ak.sum(llp_csc))
print("Total events = ",len(events))
print("Fraction of events inside CSC = ",np.round(ak.sum(llp_csc)/len(events),3))

LLP decay inside CSC =  4416
Total events =  151269
Fraction of events inside CSC =  0.029


In [203]:
events.gLLP_pt

<Array [[], [], [], [], ... [], [], [], []] type='151269 * var * float32'>

In [139]:
llp_csc = ak.any(events.gLLP_csc,axis=1)
events = events[(llp_csc==1)]


llp=ak.zip({
    'pt':events.gLLP_pt,
    'EMfrac':events.gLLP_EMFracE,
    'e':events.gLLP_e,
    'z':events.gLLP_decay_vertex_z ,
    'r':events.gLLP_decay_vertex_r,
}      
)  

llp = llp[llp.e>0]

lep=ak.zip({
    'pt':events.lepPt ,
    'eta':events.lepEta,
    'phi':events.lepPhi,
    'energy':events.lepE,
},with_name='PtEtaPhiELorentzVector',
behavior=vector.behavior    
)  

cluster= ak.zip(
    {
        "n":events.nCscRechitClusters3,
        "time":events.cscRechitCluster3Time,
        "timeSpread":events.cscRechitCluster3TimeSpread,
        "eta":events.cscRechitCluster3Eta,
        "phi":events.cscRechitCluster3Phi,        
        "NChamber":events.cscRechitCluster3NChamber,
        "MaxChamber":events.cscRechitCluster3MaxChamber,
        "MaxStation":events.cscRechitCluster3MaxStation,
        'NRechitChamberPlus11':events.cscRechitCluster3NRechitChamberPlus11,
        'NRechitChamberPlus12':events.cscRechitCluster3NRechitChamberPlus12,
        'NRechitChamberMinus11':events.cscRechitCluster3NRechitChamberMinus11,
        'NRechitChamberMinus12':events.cscRechitCluster3NRechitChamberMinus12,
        'match_MB1Seg_0p4':events.cscRechitCluster3_match_MB1Seg_0p4,
        'match_RE12_0p4':events.cscRechitCluster3_match_RE12_0p4,        
        'match_RB1_0p4':events.cscRechitCluster3_match_RB1_0p4,                
        
        "NStation10":events.cscRechitCluster3NStation10,
        "AvgStation10":events.cscRechitCluster3AvgStation10,
        "llp_match":events.cscRechitCluster3_match_gLLP,
        "dphi_cluster_MET":events.cscRechitCluster3MetXYCorr_dPhi,
    }
)

cluster_dir= ak.zip(
    {
        'pt':ak.ones_like(events.cscRechitCluster3Eta),
        "eta":events.cscRechitCluster3Eta,
        "phi":events.cscRechitCluster3Phi,
        'mass':ak.zeros_like(events.cscRechitCluster3Eta)
    },with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior
)

In [172]:
# # a = ak.flatten(llp.z[ak.any(ClusterID,axis=1)])
# <Array [908, -694, 859, ... -1.07e+03, -974] type='15 * float32'>
# b = ak.flatten(llp.z)
# <Array [621, 908, -721, ... -1.07e+03, -974] type='63 * float32'>
# ak.flatten(ak.fill_none(ak.mask(llp.z,ak.any(ClusterID,axis=1)),[0]))
# <Array [0, 908, 0, 0, ... 0, -1.07e+03, -974] type='63 * float64'>

def maskAndFill(denom,selection,value):
    numer = ak.mask(denom,selection)
    numer = ak.fill_none(numer, value) #fill none with same structure
    return ak.flatten(numer)


numer = maskAndFill(llp.e,ak.any(cluster.llp_match,axis=1),len(llp.e[0])*[0])
selection=ak.values_astype(numer>0,np.int)

h = hist.Hist("Events", hist.Bin("selection", "pass/fail", 2, 0, 2),
            hist.Bin("denom", "z", 50, 500, 1100)
         )
h.fill(selection=ak.values_astype(numer>0,np.int),
       denom=ak.flatten(llp.z)
)
h.project('selection').values()

{(): array([ 54., 126.])}

In [165]:
cluster.llp_match[3]

<Array [] type='0 * bool'>

In [164]:
np.where(ak.any(cluster.llp_match,axis=1))

(array([  0,   1,   2,   4,   5,   8,   9,  10,  11,  12,  14,  16,  17,
         19,  20,  21,  22,  23,  24,  26,  27,  29,  31,  33,  34,  35,
         36,  37,  40,  41,  43,  45,  46,  47,  49,  50,  52,  54,  56,
         57,  58,  59,  60,  62,  63,  64,  65,  66,  68,  69,  70,  72,
         73,  74,  75,  76,  77,  79,  80,  81,  83,  84,  86,  89,  92,
         93,  94,  96,  97,  99, 100, 101, 103, 105, 107, 108, 110, 111,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 126, 129,
        130, 131, 133, 136, 140, 141, 142, 144, 145, 146, 148, 149, 150,
        151, 154, 156, 157, 159, 161, 162, 163, 164, 166, 167, 169, 170,
        173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 187,
        188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 202,
        203, 204, 206, 208, 209, 210, 211, 212, 213, 215, 216, 217, 218,
        220, 221, 223, 224, 228, 230, 233, 234, 236, 237, 241, 242, 244,
        245, 246, 249, 250, 251, 253, 254, 255, 258

In [170]:
ak.fill_none(ak.mask(llp.e,ak.any(cluster.llp_match,axis=1)),[0]*2)

<Array [[47.1, 97.4], [133, ... [167, 57.6]] type='284 * var * float64'>

<Array [(47.1, 0), (97.4, 0)] type='2 * (float32, int64)'>

In [100]:
ak.sum(numer==0)+ak.sum(numer>0)

4416

In [101]:
ak.sum(numer>0)

2180

In [102]:
ak.sum(numer==0)

2236

In [40]:
ak.max(events.gLLP_decay_vertex_z)

1099.588

In [41]:
llp.z

<Array [[621], [908], ... [1.08e+03], [-571]] type='4416 * option[var * float32]'>

In [114]:
ak.any(cluster.n>1,axis=1)
llp.z[ak.any(cluster.n>1,axis=1)]

<Array [[-694], [924], [-974]] type='3 * option[var * float32]'>

In [116]:
cluster.NChamber>1

<Array [[True], [True], ... [True, True]] type='63 * option[var * bool]'>

In [198]:
abs(cluster.eta)<1.9

<Array [[True], [False], ... [True, False]] type='63 * option[var * bool]'>

In [208]:
clusterID=((cluster.NStation10>1) & (abs(cluster.eta)<1.9))|\
((cluster.NStation10==1) &(abs(cluster.AvgStation10)==4) & (abs(cluster.eta)<1.8))|\
((cluster.NStation10==1) &(abs(cluster.AvgStation10)==3) & (abs(cluster.eta)<1.6))|\
((cluster.NStation10==1) &(abs(cluster.AvgStation10)==2) & (abs(cluster.eta)<1.6))

In [228]:
clusterVeto= ((cluster.NRechitChamberPlus11<=0)&(cluster.NRechitChamberPlus12<=0)&\
              (cluster.NRechitChamberMinus11<=0)&(cluster.NRechitChamberMinus12<=0)&\
              (cluster.match_MB1Seg_0p4<=0)&(cluster.match_RE12_0p4<=0)&\
              (cluster.match_RB1_0p4<=0)
             )

In [44]:
ak.any(cluster.llp_match,axis=1)

<Array [True, True, False, ... False, False] type='4416 * ?bool'>

In [239]:
ak.sum(ClusterID)

15

In [237]:
ak.sum(ak.Array([[True,True],[False,True]]))

3

In [210]:
ClusterID= ((cluster.AvgStation10>2)&(cluster.NChamber>1))
ak.flatten(llp.z[ak.any(ClusterID,axis=1)])

<Array [908, -694, 859, ... -1.07e+03, -974] type='15 * float32'>

In [213]:
llp.z

<Array [[621], [908], ... [-1.07e+03], [-974]] type='63 * option[var * float32]'>

In [ ]:
events.gLLP_EMFracE

In [214]:
ak.values_astype(llp.z[ak.any(ClusterID,axis=1)],np.int)

<Array [[907], [-693], ... [-1074], [-973]] type='15 * option[var * int64]'>

In [50]:
numer>0

<Array [True, True, False, ... False, False] type='4416 * bool'>

<Array [1, 1, 0, 1, 0, 0, ... 1, 0, 0, 1, 0, 0] type='4416 * int64'>

In [53]:
ak.flatten(llp.z)

<Array [621, 908, -721, ... 1.08e+03, -571] type='4416 * float32'>

In [65]:
h.values()

{(): array([[57., 66., 54., 61., 72., 60., 70., 70., 50., 56., 48., 46., 54.,
         52., 61., 52., 21.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [22., 30., 20., 17., 16., 17., 20., 14.,  9., 12., 11., 14.,  7.,
          1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])}

{(): array([950., 213.])}

In [59]:
h[''].values()

IndexError: Cannot understand slice 'selection' on axis <Bin (name=selection) instance at 0x7f7c5c490320>

In [12]:
events.gLLP_pt[events.gLLP_csc==1]

<Array [33.4, 34.4, 29.6, ... 34.5, 3.49, 23.3] type='63 * float32'>

In [13]:
events.gLLP_lepdPhi

<Array [[3.09, 2.59], ... [-3.03, 1.37]] type='63 * 2 * float32'>

In [22]:
events.gLLP_decay_vertex_z!=-666

<Array [[True, False], ... [True, False]] type='63 * 2 * bool'>

In [6]:
ak.flatten(ak.values_astype(~cluster.llp_match,np.int))

<Array [0, 0, 0, 0, 1, 0, ... 0, 0, 0, 1, 0, 1] type='41 * int64'>

In [146]:
ak.flatten(cluster.n)

<Array [1, 1, 1, 2, 2, 1, ... 1, 1, 1, 1, 1, 1] type='1091 * int32'>

In [33]:
cls_lep_pair = ak.cartesian({"cls":cluster_dir,'lep':lep},axis=1,nested=True)

dphi_lep_cls = cls_lep_pair.cls.delta_phi(cls_lep_pair.lep)

In [30]:
ak.to_numpy(cluster_dir[cluster.llp_match][10])

array([(1., 1.5951381, 0.12471689, 0.)],
      dtype=[('pt', '<f4'), ('eta', '<f4'), ('phi', '<f4'), ('mass', '<f4')])

In [96]:
llp=llp[llp.e>0]
ak.flatten(llp.z[ak.num(cluster)>1])

<Array [-694, 924, -974] type='3 * float32'>

In [97]:
ak.flatten(llp.z)

<Array [621, 908, -721, ... -1.07e+03, -974] type='63 * float32'>

In [51]:
np.where(ak.num(cluster)>1)

(array([ 6, 10, 62]),)

1

In [56]:
llp.r[62]

<Array [407, 942] type='2 * float32'>

In [22]:
np.where(ak.num(cluster)>1)

(array([ 6, 10, 62]),)

In [154]:
ak.flatten(ak.values_astype(cluster.llp_match,np.int))

<Array [1, 1, 1, 1, 0, 1, ... 1, 1, 1, 1, 1, 1] type='1091 * int64'>

In [160]:
ak.fill_none(ak.flatten(dphi_lep_cls),-600)

<Array [[], [], [2.84], ... [], [], [-3.01]] type='1091 * union[var * float32, i...'>

In [163]:
ak.flatten(dphi_lep_cls[~cluster.llp_match],axis=None)

<Array [-0.314, -0.411, -1.78, ... -1.8, 0.296] type='9 * float32'>

In [164]:
ak.flatten(dphi_lep_cls[cluster.llp_match],axis=None)

<Array [2.84, 3.08, -3.12, ... 3.01, -3.01] type='179 * float32'>

In [166]:
ak.flatten(dphi_lep_cls[~cluster.llp_match],axis=None)
ak.zeros_like(ak.flatten(dphi_lep_cls[cluster.llp_match],axis=None))

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='179 * float32'>

In [87]:
ak.to_numpy(ak.flatten(dphi_lep_cls,axis=None))

array([ 2.8449643 ,  3.0809057 , -3.1249049 , -0.31369328, -3.1015875 ,
       -2.8946662 , -2.731911  ,  3.0177443 ], dtype=float32)